# __*Project Title: Cryptocurrency Prediction using LSTM Neural Network*__

### __*Importing Libraries*__

In [4]:
from yfinance import download as get
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential, load_model
from sklearn.metrics import r2_score, mean_squared_error
import plotly.graph_objs as go
import plotly.io as pio


### __*Grouping a set of coins that are closely related in terms of market dynamics or share similar price trends*__

In [5]:
coins = {1:(('Bitcoin', 'BTC'), ('Bitcoin SV', 'BSV'), ('Bitcoin Cash', 'BCH')),
         2:(('Ethereum', 'ETH'), ('Ethereum Classic', 'ETC'), ('Cardano', 'ADA')),
         3:(('Litecoin', 'LTC'), ('Dogecoin', 'DOGE'), ('Bitcoin Gold', 'BTG')),
         4:(('Ripple', 'XRP'), ('Stellar','XLM'),  ('EOS','EOS')),
         5:(('Binance Coin', 'BNB'), ('Huobi Token', 'HT'),  ('OKB', 'OKB'))}


### __*Creating and returning model*__

In [6]:
def getModel(input_shape):
  model = Sequential()
  model.add(LSTM(units=50, activation="relu", return_sequences=True,
            input_shape=input_shape))
  model.add(Dropout(0.2))
  model.add(LSTM(units=60, activation="relu", return_sequences=True))
  model.add(Dropout(0.3))
  model.add(LSTM(units=80, activation="relu", return_sequences=True))
  model.add(Dropout(0.4))
  model.add(LSTM(units=120, activation="relu"))
  model.add(Dropout(0.5))
  model.add(Dense(units=1))
  model.compile(optimizer="adam", loss="mean_squared_error")
  return model



### __*Splitting data into x data and y data*__

In [7]:
def processData(data, time_period):
  x_data, y_data = [], []
  for i in range(time_period, data.shape[0]):
    x_data.append(data[i - time_period:i, 0])
    y_data.append(data[i, 0])
  return np.array(x_data), np.array(y_data)


### __*Function used Getting data processing it*__


In [8]:
def getData(coin):
  data = {}
  data['data'] = pd.DataFrame(get(f"{coin[1]}-USD")["Adj Close"])
  data['training'] = pd.DataFrame(
    data['data'][:int(np.floor(len(data['data']) * 0.9))])
  data['testing'] = pd.DataFrame(
    data['data'][int(np.floor(len(data['data']) * 0.9)) - 30:])
  data['scaler'] = MinMaxScaler(feature_range=(-1, 1))
  data['x_train'], data['y_train'] = processData(
    data['scaler'].fit_transform(data['training']), 30)
  data['x_test'], data['y_test'] = processData(
    data['scaler'].fit_transform(data['testing']), 30)
  return data


### *__Getting data for all coins__*

In [9]:
data = {}
for setno, cset in coins.items():
  for coin in cset:
    print(coin[0])
    data[coin[0]] = getData(coin)


Bitcoin
[*********************100%***********************]  1 of 1 completed
Bitcoin SV
[*********************100%***********************]  1 of 1 completed
Bitcoin Cash
[*********************100%***********************]  1 of 1 completed
Ethereum
[*********************100%***********************]  1 of 1 completed
Ethereum Classic
[*********************100%***********************]  1 of 1 completed
Cardano
[*********************100%***********************]  1 of 1 completed
Litecoin
[*********************100%***********************]  1 of 1 completed
Dogecoin
[*********************100%***********************]  1 of 1 completed
Bitcoin Gold
[*********************100%***********************]  1 of 1 completed
Ripple
[*********************100%***********************]  1 of 1 completed
Stellar
[*********************100%***********************]  1 of 1 completed
EOS
[*********************100%***********************]  1 of 1 completed
Binance Coin
[*********************100%*****************

### *__Concating data of coins in same group and traning model and saving model__*

In [15]:
for setno, cset in coins.items():
  print(f"Set {setno} Started")
  x_train, y_train = [], []
  for coin in cset:
    x_train.extend(data[coin[0]]['x_train'])
    y_train.extend(data[coin[0]]['y_train'])
  x_train, y_train = np.array(x_train), np.array(y_train)
  model = getModel((30,1))
  model.fit(x_train, y_train, epochs=108)
  model.save(f'models/model{setno}.h5')
  print(f"Set {setno} completed")


Set 1 Started
Epoch 1/108
187/187 [==============================] - 23s 85ms/step - loss: 0.0576
Epoch 2/108
187/187 [==============================] - 16s 85ms/step - loss: 0.0241
Epoch 3/108
187/187 [==============================] - 14s 77ms/step - loss: 0.0207
Epoch 4/108
187/187 [==============================] - 16s 85ms/step - loss: 0.0186
Epoch 5/108
187/187 [==============================] - 14s 76ms/step - loss: 0.0176
Epoch 6/108
187/187 [==============================] - 14s 72ms/step - loss: 0.0152
Epoch 7/108
187/187 [==============================] - 14s 73ms/step - loss: 0.0147
Epoch 8/108
187/187 [==============================] - 14s 75ms/step - loss: 0.0137
Epoch 9/108
187/187 [==============================] - 14s 74ms/step - loss: 0.0127
Epoch 10/108
187/187 [==============================] - 14s 75ms/step - loss: 0.0116
Epoch 11/108
187/187 [==============================] - 14s 73ms/step - loss: 0.0111
Epoch 12/108
187/187 [==============================] - 14s 

### *__Preduction of testing data and ploting on graph and saving it__*

In [18]:
for setno, cset in coins.items():
  model = load_model(f"models/model{setno}.h5")
  for coin in cset:
    data[coin[0]]['y_predict'] = model.predict(data[coin[0]]['x_test'])
    data[coin[0]]['rmse'] = np.sqrt(mean_squared_error(
        data[coin[0]]['y_test'], data[coin[0]]['y_predict']))
    data[coin[0]]['accuracy'] = r2_score(
      data[coin[0]]['y_test'], data[coin[0]]['y_predict']) * 100
    data[coin[0]]['predict'] = pd.DataFrame(data[coin[0]]['scaler'].inverse_transform(data[coin[0]]['y_predict']), index=data[coin[0]]['testing'].index[30:], columns=['Adj Close'])
    plot = go.Figure()
    plot.add_trace(go.Scatter(
        name="Training Data",
        x=data[coin[0]]['training'].index.date,
        y=data[coin[0]]['training']['Adj Close'],
        mode="lines",
        # line_color='li',
    ))
    plot.add_trace(go.Scatter(
        name='Testing Data',
        x=data[coin[0]]['testing'].index[30:].date,
        y=data[coin[0]]['testing']['Adj Close'][30:],
        mode="lines",
        line_color='green',
    ))
    plot.add_trace(go.Scatter(
        name='Predicted Data',
        x=data[coin[0]]['predict'].index.date,
        y=data[coin[0]]['predict']['Adj Close'],
        mode="lines",
        line_color='red',
    ))
    plot.update_layout(
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=True,
        xaxis_range=[data[coin[0]]['training'].index[-(int(np.floor(len(data[coin[0]]['predict']) * 0.5)))], data[coin[0]]['predict'].index[-1]],
        legend=dict(orientation="h", yanchor="bottom", y=-0.7, xanchor="left", x=0),
        height=550,
        width=700,
        yaxis=dict(tickformat=',d')
    )
    plot.update_xaxes(rangeselector=dict(
        buttons=list([
            dict(count=1, label="MONTH", step="month", stepmode="backward", ),
            dict(count=6, label="6 MONTH", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="YEAR", step="year", stepmode="backward"),
            dict(label='ALL', step="all")
        ]), y=1.1,
    ))
    pio.write_json(plot, file=f"plots/{coin[0].replace(' ', '_')}.json")

5/5 [==============================] - 0s 23ms/step


### *__Getting Accuracy & Root mean square error__*

In [19]:
print('Accuracy percentage = ', np.average(
    [arc['accuracy'] for arc in data.values()]))
print('Root mean square error = ', np.average(
    [arc['rmse'] for arc in data.values()]))


Accuracy percentage =  88.75928509591448
Root mean square error =  0.12702707795912385
